In [1]:
#import pandas and read in csv
import pandas as pd

In [3]:
sfr_df = pd.read_csv("/content/sfr_cleaned_only_important_columns.csv")
sfr_df.head()


,Bedrooms,Total Baths,SqFt,Acres,Sold Price,List Year,List Month,Age of Property,Rate,Lat,...,City_Raleigh,City_Rocky Mount,City_Sanford,City_Selma,City_Smithfield,City_Wake Forest,City_Wendell,City_Willow Springs,City_Youngsville,City_Zebulon
0,3,1,-1.569749,-0.160903,54000,1,2,2.043593,-1.029184,-0.753785,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,3,2,-1.096891,-0.160903,57500,0,12,1.509363,-1.115292,0.241470,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,-1.626796,-0.160903,60000,1,2,2.488785,-1.115292,0.605112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1,-1.482277,-0.160903,62000,1,3,2.132632,-1.023443,-0.803133,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,1,0.109973,-0.160903,63000,0,11,1.954555,-1.258804,0.620745,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
sfr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53564 entries, 0 to 53563
Data columns (total 71 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Bedrooms             53564 non-null  int64  
 1   Total Baths          53564 non-null  int64  
 2   SqFt                 53564 non-null  float64
 3   Acres                53564 non-null  float64
 4   Sold Price           53564 non-null  int64  
 5   List Year            53564 non-null  int64  
 6   List Month           53564 non-null  int64  
 7   Age of Property      53564 non-null  float64
 8   Rate                 53564 non-null  float64
 9   Lat                  53564 non-null  float64
 10  Lon                  53564 non-null  float64
 11  spatial_feature_1    53564 non-null  float64
 12  spatial_feature_2    53564 non-null  float64
 13  spatial_feature_3    53564 non-null  float64
 14  spatial_feature_4    53564 non-null  float64
 15  spatial_feature_5    53564 non-null 

In [5]:
corr_matrix = sfr_df.corr()
corr_matrix["Sold Price"].sort_values(ascending=False)

,Sold Price
Sold Price,1.000000
SqFt,0.750650
Total Baths,0.622471
Bedrooms,0.484341
City_Cary,0.220977
...,...
City_Rocky Mount,-0.133444
City_Clayton,-0.133974
City_Burlington,-0.163441
Age of Property,-0.273170


In [6]:
sfr_df['Sold Price'].value_counts()

,count
Sold Price,
450000,616
350000,607
400000,571
375000,501
500000,488
...,...
882500,1
878500,1
151500,1


In [7]:
#identifying our x and y variables and splitting the data
from sklearn.model_selection import train_test_split

X = sfr_df.drop(columns='Sold Price')
y = sfr_df['Sold Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [8]:
#!pip install category_encoders
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.4 MB/s eta 0:00:00


In [9]:
#Build a pipeline of training
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

estimators = [

    ('clf', XGBRegressor(objective='reg:squarederror', random_state=8))
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('clf',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=8, ...))])

In [10]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.1 MB/s eta 0:00:00


In [11]:
#Set up hyperparameter tuning
from skopt import BayesSearchCV
from skopt.space import Real, Integer

search_space = {
    'clf__max_depth': Integer(3, 12),
    'clf__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    'clf__colsample_bylevel': Real(0.6, 1.0),
    'clf__colsample_bynode' : Real(0.6, 1.0),
    'clf__reg_alpha': Real(0.0, 3.0),
    'clf__reg_lambda': Real(0.0, 5.0),
    'clf__gamma': Real(0.0, 5.0),
    'clf__min_child_weight': Integer(2, 10),
    'clf__n_estimators': Integer(50, 600)
}
# Perform cross-validation with early stopping
bayes_search = BayesSearchCV(pipe, search_space, cv=8, n_iter=50, scoring='r2', random_state=8)

In [12]:
#Train the XGBoost model
bayes_search.fit(X_train, y_train)

BayesSearchCV(cv=8,
              estimator=Pipeline(steps=[('clf',
                                         XGBRegressor(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,
                                                      feature_types=None,
                                                      gamma=None,
                                                      grow_policy=None,
                                                      importance_type=None,
                                                      interaction_constraints=N...
                             'clf__min_child_weight': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'clf__n_estimators': Integer(low=50, high=600, prior='uniform', transform='normalize'),
                             'clf__reg_alpha': Real(low=0.0, high=3.0, prior='uniform', transform='normalize'),
                             'clf__reg_lambda': Real(low=0.0, high=5.0, prior='uniform', transform='normalize'),
                             'clf__subsample': Real(low=0.6, high=1.0, prior='uniform', transform='normalize')})

In [13]:
#Evaluate the model and make predictions
best_model = bayes_search.best_estimator_

In [14]:
bayes_search.best_score_

0.8943478956915709

In [15]:
bayes_search.score(X_test, y_test)

0.8974277332683237

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred_train = bayes_search.predict(X_train)

mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

print(f"Mean Squared Error (MSE) for trained data: {mse_train}")
print(f"Mean Absolute Error (MAE) for trained data: {mae_train}")

Mean Squared Error (MSE) for trained data: 1171949762.6859877
Mean Absolute Error (MAE) for trained data: 24463.335774052095


In [17]:
#make predictions with testing data
y_pred = bayes_search.predict(X_test)

In [18]:
#ensure all data in y_pred are integers
y_pred = y_pred.round().astype(int)

In [19]:
#score model using MSE and MAE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

Mean Squared Error (MSE): 3500288309.0589004
Mean Absolute Error (MAE): 40049.72967422757
